# Base de données relationnelles - DDL et DML

Nous abordons les parties du **langage SQL** (*Structured Query Langage*) qui permettent de spécifiér le **schéma relationnel** d'une base de données; il y en a deux:
- Une pour la **définition des données** abrégé en *DDL* pour *Data Definition Language*
    
    - *mots clés*: `CREATE TABLE`, `DROP TABLE`, `ALTER TABLE`.

- L'autre pour leur **manipulation** abrégé en *DML* pour *Data Manipulation Language*.
    
    - *mots clés*: `INSERT INTO`, `UPDATE` et `DELETE`.

**Utiliser la table des matières pour «naviguer» dans le document**

## Création de tables

La syntaxe SQL pour la création d'une table est de la forme:

```sql
CREATE TABLE nom_table (
    nom_champ1  domaine  [contraintes],
    nom_champ2  domaine  [contraintes],
    ...
    [contrainte1,
    contrainte2,
    ...]
);
```

Les crochets `[]` indiquent une partie optionnelle. Attention aux `,` qui terminent chaque ligne **sauf** la dernière.

Une contrainte **sur un seul attribut** (champ) peut être indiqué à la suite du domaine. Par exemple:

        ...,
        nom VARCHAR(20) NOT NULL,
        ...
        
indique que l'attribut `nom` est une chaîne de caractère de taille max 20 caractères qui possède une contrainte de non nullité qui signifie qu'il n'est pas possible d'avoir une ligne (entité) dont le nom ne soit pas précisé.

Lorsque une contrainte porte sur **plusieurs attributs**, il est possible de la placer après toutes les déclarations d'attributs; par exemple:

        CREATE TABLE personnes(
            id ...,
            nom ...   ,
            prenom ...,
            maman ...,
            ...,
            PRIMARY KEY id,
            UNIQUE (nom, prenom),
            FOREIGN KEY maman REFERENCES personnes (id),
            ...
        );

La contrainte d'unicité - `UNIQUE` signifie que deux personnes ne peuvent être «homonyme» dans cette table.

L'attribut `id` sert de *clé primaire* (il ne peut y en avoir qu'une par table), et `maman` est une *clé étrangère* qui fait référence à une personne de cette table. Notez que pour un clé étrangère, il est obligatoire de citer la table à laquelle elle fait référence.

Bien sûr, comme ces clés portent sur un seul attribut, on peut les mettres sur la même ligne que l'attribut visé:

        CREATE TABLE personnes(
            id ... PRIMARY KEY,
            nom ...   ,
            prenom ...,
            maman ... REFERENCES personnes (id),
            ...,
            UNIQUE (nom, prenom),
            ...
        );

ce qui allège un peu.

### Suppression d'une table - \[*drop table*\]

C'est simplissime ...
```sql
DROP TABLE nom_table;
```
... mais cela peut échouer. En bref, si la table qu'on souhaite supprimée est référencée par les lignes d'une autre table (clé étrangère vers la table à supprimer), l'opération échoue. On peut forcer la suppression en cascade:

```sql
DROP TABLE nom_table CASCADE;
```
ce qui a pour effet de supprimer la ou les tables qui référence celle qu'on souhaite supprimer.

[En savoir plus...](https://docs.postgresql.fr/13/ddl-depend.html)

### Modification d'une table - \[alter table\]

Nous ne détaillerons pas cette partie du langage. Voici un simple exemple:

```sql
ALTER TABLE membres ADD COLUMN email VARCHAR(50);
```

Ajoute le champ `email` à la table `membres`.

[En savoir plus...](https://docs.postgresql.fr/13/ddl-alter.html)

## Domaines de données les plus courants

La normes SQL prévoit les **domaines** (ou types) suivants:

### Numériques

**Entiers** (signés): 
- `INTEGER` alias `INT` (codé sur 4 octets), 
- mais aussi `SMALLINT` (2 octets) et `BIGINT` (8 octets).

**Décimaux**:
- **exactes**: `DECIMAL(f, d)` alias `NUMERIC(f, d)`.
    - si `f` et `d` ne sont pas précisés - `DECIMAL` ou `NUMERIC` - la précision est maximale et dépend de la machine utilisée,
    - `f` désigne le nombre de chiffres significatifs et `d` le nombre de chiffres de la partie décimale.
      
      *ex:* pour représenter 543.21 on a besoin de 5 chiffres significatifs et de 2 pour la partie décimale - `DECIMAL(5,2)`.
- **approchés**: `REAL` qui représente un flottant codé sur 4 octets.

[Pour en savoir plus (postgresql)](https://docs.postgresql.fr/13/datatype-numeric.html)

### Chaînes de caractères

- `CHAR(nb_car)`: chaîne de **taille fixe** précisé par `nb_car`,
- `VARCHAR(nb_car_max)`: chaîne de **taille variable** mais dont le nombre maximum de caractères est précisé,
- `TEXT`: chaîne de longueur arbitraire.

*Note*: si une chaîne dépasse la taille maximum, elle est souvent *tronquée* sans pour autant produire d'erreur.

[Pour en savoir plus (postgresql)](https://docs.postgresql.fr/13/datatype-character.html)

### Dates/heures

- `DATE`: représente une **date** (jour-mois-année). 
  
  Par exemple, pour spécifier *le samedi 5 décembre 2020*, on peut écrire `'2020-12-05'` (format ISO - *year-month-day*),
- `TIME`: réprésente un **horaire** *sans précision d'un fuseau horaire* (*time zone*). 
  
  Par exemple `'08:37:42.348` (format ISO) signifie 8h37 et 42 secondes et 348 millièmes de secondes. Bien sûr, on peut être moins précis: `'08:37'`,
- `TIMESTAMP`: réprésente un **horodatage** (date + horaire) *sans précision d'un fuseau horaire*. 

  Par exemple `'2020-12-05 08:37'`.

Il est possible de préciser un horaire ou un horodatage avec un **fuseau horaire** en suffixant le type avec `TZ` (pour *time zone*). Par exemple `'2020-12-05 08:37+01'` ou `'2020-12-05 08:37 Europe/Paris'` désigne un horodatage valable pour la France à l'heure d'hivers...

Il existe aussi des **valeurs spéciales** comme `'now'` (TIMESTAMP), `'today'`, `'tomorrow'`, `'yesterday'` (DATE) etc. qui se rapportent au moment où la transaction SQL est réalisée (INSERT INTO ...).

[Pour en savoir plus (postgresql)](https://docs.postgresql.fr/13/datatype-datetime.html)

### Autres types

Il existe bien d'autres types mais *ils dépendent souvent du SGBDR utilisé*. 

Pour **Postgresql**, citons:
- `SERIAL`: type numérique entier qui est automatiquement incrémenté lors de l'insertion d'une entité dans une table; s'utilise fréquemment pour une clé primaire numérique,
- le type booléen: `BOOLEAN` avec ces valeurs `true/false` ou `on/off` ou `1/0` ...
- Les types «géométriques»: `POINT`, `LINE`, `POLYGON`, `PATH` etc.
- la possibilité de créer ses propres types - `CREATE TYPE nom_type AS (...);`

[Pour en savoir plus (postgresql)](https://docs.postgresql.fr/13/datatype.html)

## Spécification des contraintes d'intégrités

[*Vue d'ensemble*](https://docs.postgresql.fr/13/ddl-constraints.html)

### Clé primaire - \[*primary key*\]

Si elle porte sur un seul attribut, il suffit d'écrire `PRIMARY KEY` dans la zone [options] de la ligne,

Sinon, on la place sur sa ligne propre *après les déclarations d'attributs*: `PRIMARY KEY (attr1, attr2, ...)`

[En savoir plus...](https://docs.postgresql.fr/13/ddl-constraints.html#DDL-CONSTRAINTS-PRIMARY-KEYS)

### Clé étrangère - \[*foreign key*\]

Si elle porte sur un seul attribut, il suffit d'écrire `REFERENCES table_visé (nom_attr)`

Si elle porte sur plusieurs attributs (moins fréquent), on place sa déclaration après celle des attributs:

    FOREIGN KEY (attr1, attr2, ...) REFERENCES table_visé (attr1', attr2',...)

Le «prime» de `attr'` signifie «le nom de l'attribut dans la `table_visé`» (qui peut être identique ou différent de celui de la table courante).

[En savoir plus...](https://docs.postgresql.fr/13/ddl-constraints.html#DDL-CONSTRAINTS-FK)

### de Vérification - \[*check*\]

Il est possible de s'assurer que la valeur d'un attribut respecte certaines conditions.

Par exemple, supposons avoir un attribut *note* de type `DECIMAL(3,1)`. Rien ne garantie que la note soit entre 0 et 20. Pour s'assurer que c'est bien le cas, on pourrait utiliser la contrainte de vérification:

    CHECK (note >= 0 AND note <= 20)

[En savoir plus...](https://docs.postgresql.fr/13/ddl-constraints.html#DDL-CONSTRAINTS-CHECK-CONSTRAINTS)

### de non nullité - \[*not null*\]

Par défaut, un attribut peut ne recevoir «aucune valeur» ce qui correspond à la valeur spéciale `NULL` en SQL.

Pour préciser qu'un attribut DOIT être renseigné, on peut utiliser la contrainte `NOT NULL` qu'on place à la suite du type de l'attribut.

[En savoir plus...](https://docs.postgresql.fr/13/ddl-constraints.html#id-1.5.4.6.6)

### d'unicité - \[*unique*\]

Il peut parfois être souhaitable d'imposer qu'un attribut ou une combinaison d'attributs soit unique dans une table. Ainsi, deux entités distinctes ne peuvent pas avoir une même valeur pour un attribut `UNIQUE`.

Par exemple, si l'attribut `no_tel` ne doit pas pouvoir prendre deux fois la même valeur dans une table:

        ...
        no_tel CHAR(10) NOT NULL UNIQUE,
        ...

Une clé primaire vérifie automatiquement cette contrainte donc ne l'utiliser que pour un attribut ou une combinaison d'attributs qui n'ont pas la contrainte clé primaire.

Pour une combinaison d'attributs, indiquer la contrainte sur sa propore ligne:

        ...
        UNIQUE (nom, prenom)
        ...

[En savoir plus...](https://docs.postgresql.fr/13/ddl-constraints.html#DDL-CONSTRAINTS-UNIQUE-CONSTRAINTS)

## Manipulation de données

### [Insertion](https://docs.postgresql.fr/13/dml-insert.html) - \[*insert into ... values ...*\]

Pour insérer des données, il y a deux syntaxes (préférer la seconde pour l'écriture d'un script sql):

- **Implicite**: les valeurs doivent-être indiquées dans le même ordre que celui de la déclaration des attributs

```sql
INSERT INTO nom_table VALUES (v_attr1, v_attr2, ...);
```

- **Explicite**: les attributs visés (on peut ne pas en préciser certains s'il peuvent être laissés vides ou s'ils possède une valeur par défaut) sont précisés et les valeurs données dans le même ordre.

```sql
INSERT INTO nom_table (nom_attr2, nom_attr5, ...) VALUES (v_attr2, v_attr5, ...);
```

Il est possible d'insérer plusieurs entités en séparant les n-uplets de valeurs avec une `,`.

Noter que si vous utilisez un attribut de type `SERIAL` (entier auto incrémenté) comme clé primaire, il faut utiliser la syntaxe explicite sans mentionner cet attribut (puisqu'il est calculé automatiquement).

[En savoir plus...](https://docs.postgresql.fr/13/dml-insert.html)

### [Actualisation](https://docs.postgresql.fr/13/dml-insert.html) - \[*update ... set ...*\]

Donnons un simple exemple: on veut diminuer un attribut `prix` de 10% d'une table **produits**:

```sql
UPDATE produits SET prix = prix * 0.9;
```

Il est possible d'ajouter une condition pour n'appliquer ce changement qu'à certaines lignes; par exemple:

```sql
UPDATE produits SET prix = prix * 0.9 where en_promo IS 'true';
```
en supposant qu'il y ait un attribut booléen `en_promo`.

[En savoir plus...](https://docs.postgresql.fr/13/dml-update.html)

### Suppression - \[*delete from ...*\]

Deux exemples:
- suppression de toutes les lignes:
  ```sql
  DELETE FROM produits;
  ```
- suppression de certaines lignes:
  ```sql
  DELETE FROM produits WHERE prix < 5;
  ```

*Note*: la suppression peut échouer si la ligne qu'on souhaite supprimée *est référencée* par une ligne d'une autre table (contrainte de clé étrangère). Dans ce cas, il faut commencer par supprimer toutes les lignes de toutes les tables qui références la ligne visée...

[En savoir plus...](https://docs.postgresql.fr/13/dml-delete.html)